## Prepare imports

In [1]:
import os
import torch
import numpy as np
import torch.nn as nn

from load_models import load_encoder, load_vae
from torch_geometric.data import Data, Batch

from omegaconf import OmegaConf
import torch
import time
import statistics

In [2]:
DEVICE = "cuda:0"


class TopologicalModelVAE:
    def __init__(self, encoder, vae) -> None:
        self.encoder = encoder
        self.vae = vae
        # self.vae.model.eval()

    def sample(self, B, N):
        """
        The way we expect the input
        B is the number of point clouds
        N is the number of points per cloud.
        _, out_pc = model.sample(B, N)
        """
        ect_samples = self.vae.model.sample(B, "cuda:0")

        # Rescale to 0,1
        ect_samples = (ect_samples + 1) / 2

        vae_pointcloud = self.encoder(ect_samples).view(B, N, 3)
        return ect_samples, vae_pointcloud

    @torch.no_grad()
    def reconstruct(self, x, num_points=2048):
        """
        Takes in a pointcloud of the form BxPxD
        and does a full reconstruction into
        a pointcloud of the form BxPxD using our model.

        We follow the PointFlow signature to make it compatible with
        their framework.
        """

        x_means = x.mean(axis=1, keepdim=True)
        x = x - x_means

        x_norms = torch.norm(x, dim=2).max(axis=1)[0].reshape(-1, 1, 1)
        x = x / x_norms

        # Reshape into a torch_geometric batch

        batch = Batch.from_data_list([Data(x=pts.view(-1, 3)) for pts in x])

        batch = batch.to(DEVICE)
        ect = self.encoder.layer(batch, batch.batch)
        # encoder_pointcloud = self.encoder_model(ect)
        ect = 2 * ect - 1
        reconstructed_ect, _, _, _ = self.vae(ect.unsqueeze(1))

        # Rescale to 0,1
        reconstructed_ect = (reconstructed_ect + 1) / 2

        vae_pointcloud = self.encoder(reconstructed_ect).view(-1, num_points, 3)

        vae_pointcloud = vae_pointcloud * x_norms
        vae_pointcloud = vae_pointcloud + x_means

        assert vae_pointcloud.shape == x.shape

        return vae_pointcloud


class TopologicalModelEncoder:
    def __init__(self, encoder_model) -> None:
        super().__init__()
        self.encoder_model = encoder_model

    def reconstruct(self, x, num_points=2048):
        """
        Takes in a pointcloud of the form BxPxD
        and does a full reconstruction into
        a pointcloud of the form BxPxD using our model.

        We follow the PointFlow signature to make it compatible with
        their framework.
        """

        x_means = torch.mean(x, axis=-2)
        x = x - x_means.unsqueeze(1)
        # print("-------POINT CLOUD MEANS ======")
        # print(x_means)

        x_norms = torch.norm(x, dim=-1).max(axis=1)[0].reshape(-1, 1, 1)
        x = x / x_norms

        # Reshape into a torch_geometric batch

        batch = Batch.from_data_list([Data(x=pts.view(-1, 3)) for pts in x])

        batch = batch.to(DEVICE)
        ect = self.encoder_model.layer(batch, batch.batch)
        encoder_pointcloud = self.encoder_model(ect).view(-1, num_points, 3)

        # print("-------DECODED POINT CLOUD BEFORE NORMALIZATION ======")
        # print(encoder_pointcloud.mean(dim=-2))
        # print("-------======")

        encoder_pointcloud = encoder_pointcloud * x_norms
        encoder_pointcloud = encoder_pointcloud + x_means.unsqueeze(1)

        # print("-------DECODED AFTER NORMALIZATION ======")
        # print(encoder_pointcloud.mean(dim=-2))
        # print("---------------------------------------------")
        # raise "bye"

        assert encoder_pointcloud.shape == x.shape

        return encoder_pointcloud




In [7]:
encoder_model = load_encoder("./trained_models/ectencoder_shapenet_chair.ckpt")
vae = load_vae("./trained_models/vae_shapenet_car.ckpt")
# model = TopologicalModelVAE(encoder_model, vae)

print(vae.hparams)



"ectconfig":     namespace(num_dims=3, num_thetas=96, bump_steps=96, r=4.5, ect_type='points', device='cuda:0', num_features=3, normalized=True)
"learning_rate": 0.0005
"max_epochs":    200
"vaeconfig":     namespace(in_channels=1, latent_dim=256, img_size=96, save_name='vae_shapenet_car.ckpt')


# Encoder times for decoding and ECT

In [4]:
# Warm-up runs
x = torch.rand(size=(2,1,96,96)).cuda()
for _ in range(10):
    _ = encoder_model(x)
# Multiple iterations
num_iterations = 10000
inference_times = []
for _ in range(num_iterations):
    start_time = time.time()
    _ = encoder_model(x)
    end_time = time.time()
    inference_times.append(end_time - start_time)
average_inference_time = statistics.mean(inference_times)
print(f"Average inference time: {average_inference_time:.8f} seconds")

Average inference time: 0.00039849 seconds


In [5]:
# Warm-up runs
x = torch.rand(size=(2,1,96,96)).cuda()
for _ in range(10):
    recon,_,_,_ = vae.model(x)
    res = encoder_model((recon + 1) / 2)
# Multiple iterations
num_iterations = 10000
inference_times = []
for _ in range(num_iterations):
    start_time = time.time()
    recon,_,_,_ = vae.model(x)
    res = encoder_model((recon + 1) / 2)
    end_time = time.time()
    inference_times.append(end_time - start_time)
average_inference_time = statistics.mean(inference_times)
print(f"Average inference time: {average_inference_time:.8f} seconds")

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [205]:
# Warm-up runs
for _ in range(10):
    samples = vae.model.sample(10,"cuda:0")
    res = encoder_model(samples)
# Multiple iterations
num_iterations = 10000
inference_times = []
for _ in range(num_iterations):
    start_time = time.time()
    samples = vae.model.sample(10,"cuda:0")
    res = encoder_model((samples + 1) / 2)
    end_time = time.time()
    inference_times.append(end_time - start_time)
average_inference_time = statistics.mean(inference_times)
print(f"Average inference time: {average_inference_time:.8f} seconds")

Average inference time: 0.00109125 seconds
